### old

In [4]:
# # 載入IMDB資料集
# from datasets import load_dataset
# dataset = load_dataset("imdb")

In [6]:
# 下載並儲存 IMDB 資料集
# import os
# import json

# local_dir = "./"

# local_file = "./imdb_dataset.jsonl"

# # 合併所有分割的資料並儲存至單一檔案
# with open(local_file, "w", encoding="utf-8") as f:
#     for split in dataset.keys():
#         for record in dataset[split]:
#             record["split"] = split  # 添加分割標籤
#             f.write(json.dumps(record, ensure_ascii=False) + "\n")



In [1]:
import os
import json
from datasets import Dataset, DatasetDict
from datasets import load_dataset


# 讀取本地 IMDB 資料集並轉換為 DatasetDict
def read_local_imdb_dataset():
    local_file = "C:/Users/PARALELL/Desktop/code/DistilBert BERT/6/imdb_dataset.jsonl"

    if not os.path.exists(local_file):
        raise FileNotFoundError(f"The file {local_file} does not exist. Please run the download function first.")

    data_by_split = {"train": [], "test": [], "unsupervised": []}
    with open(local_file, "r", encoding="utf-8") as f:
        for line in f:
            record = json.loads(line)
            split = record.pop("split")
            data_by_split[split].append(record)

    dataset_dict = DatasetDict({
        split: Dataset.from_dict({
            "text": [item["text"] for item in records],
            "label": [item["label"] for item in records]
        }) for split, records in data_by_split.items()
    })

    return dataset_dict
imdb = read_local_imdb_dataset()

#### 模型相關

##### old V3

In [6]:
# 檢查 CUDA 版本和 PyTorch 版本
import torch
cuda_version = torch.version.cuda
# print("cuda_version :", cuda_version)
# print("torch.__version__ :", torch.__version__)

# 設置設備，0 表示第一個 GPU，-1 表示 CPU
device = 0 if torch.cuda.is_available() else -1
# print("device :", device)

# =================================================================================================================
# V3
def generate_text(prompt, max_new_tokens=2048):
    """
    使用 Llama 模型生成文本，僅輸出重寫後的句子或摘要。

    Args:
        prompt (str): 輸入的提示語。
        max_new_tokens (int): 生成的最大新 token 數量。

    Returns:
        str: 模型生成的文本。
    """
    # Tokenize 輸入
    inputs = tokenizer(prompt, return_tensors='pt').to('cuda')

    # 定義生成參數
    generation_config = {
        "max_new_tokens": max_new_tokens,
        "temperature": 0.5,  # 調低溫度以提高一致性
        "top_p": 0.95,
        "repetition_penalty": 1.2,
        "eos_token_id": tokenizer.eos_token_id,
        "pad_token_id": tokenizer.eos_token_id,  # 避免 pad token 問題
        "no_repeat_ngram_size": 3,  # 避免重複 n-gram
    }

    # 生成回應
    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            **generation_config
        )

    # 解碼回應
    decoded_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return decoded_output,decoded_output.strip()

def create_polarity_reversed_sentence(prompt_polarity_reversed,original_sentence):
    """
    將正向句子翻轉為負向，或負向句子翻轉為正向。
    要求模型僅輸出重寫後的句子，不要重複原始內容。
    """
    prompt = prompt_polarity_reversed
    prompt = prompt.format(original_sentence=original_sentence)
    return generate_text(prompt)


def create_double_negation_sentence(original_sentence):
    """
    將句子改寫為包含雙重否定的句子，同時保持原本的正負向情感不變。
    要求模型僅輸出重寫後的句子，不要重複原始內容。
    """
    prompt = (
        "You are a helpful assistant. Rewrite the given movie review sentence to include double negation "
        "while keeping the same sentiment. Output only the rewritten sentence without including the original sentence."
        f"\n\nOriginal: {original_sentence}\nRewritten:"
    )
    return generate_text(prompt)

def create_noisy_sentence(original_sentence):
    """
    將句子改寫為含有拼字錯誤、口語化字詞、網路用語和雜訊的版本，並保持原本情緒。
    要求模型僅輸出重寫後的句子，不要重複原始內容。
    """
    prompt = (
        "You are a helpful assistant. Rewrite the given movie review sentence by adding misspellings, slang, "
        "internet abbreviations (e.g., 'lol', 'btw'), and random filler words while maintaining its original sentiment. "
        "Output only the rewritten sentence without including the original sentence."
        f"\n\nOriginal: {original_sentence}\nRewritten:"
    )
    return generate_text(prompt)     

In [7]:
promptV2 = (
        "You are tasked with reversing the sentiment of sentences. Transform positive sentences into negative ones, and negative sentences into positive ones. "
        "Make sure to maintain the grammatical correctness and original style of the sentence, but change the sentiment completely.\n\n"
        "Examples:\n"
        "1. Original: I absolutely loved this movie! The acting was superb and the storyline was captivating.\n"
        "   Rewritten: I absolutely hated this movie! The acting was terrible and the storyline was boring.\n"
        "2. Original: The hotel was dirty and the staff were rude.\n"
        "   Rewritten: The hotel was clean and the staff were friendly and accommodating.\n\n"
        "Now reverse the sentiment of the following sentence:\n\n"
        "Original: {original_sentence}\nRewritten:"
    )
promptV3 = (
        "You are tasked with reversing the sentiment of sentences. Transform positive sentences into negative ones, and negative sentences into positive ones. "
        "Keep the main topic and structure of the original sentence unchanged. Only modify the sentiment words and expressions.\n\n"
        "Examples:\n"
        "1. Original: I absolutely loved this movie! The acting was superb and the storyline was captivating.\n"
        "   Rewritten: I absolutely hated this movie! The acting was terrible and the storyline was boring.\n"
        "2. Original: The hotel was dirty and the staff were rude.\n"
        "   Rewritten: The hotel was clean and the staff were friendly and accommodating.\n\n"
        "Now reverse the sentiment of the following sentence while keeping its topic and structure intact:\n\n"
        "Original: {original_sentence}\nRewritten:"
    )


prompt_polarity_reversed = promptV3

In [ ]:
summarizedReview = imdb_summarized_train["summarized_text"][1]
polarity_reversed_result = create_polarity_reversed_sentence(prompt_polarity_reversed, summarizedReview)
print(polarity_reversed_result)

In [ ]:
summarizedReview = imdb_summarized_train["summarized_text"][1]
double_negation_result = create_double_negation_sentence(summarizedReview)
print(double_negation_result)

### 讀取檔案

##### 讀取摘要後內容

In [1]:
import pandas as pd
imdb_summarized_train = pd.read_csv("./data/2. summarized/2024.12.18_23.43.16_imdb_train.csv",index_col=0)
imdb_summarized_test = pd.read_csv("./data/2. summarized/2024.12.24_22.49.05_imdb_test.csv",index_col=0)
imdb_summarized_val = pd.read_csv("./data/2. summarized/2024.12.24_20.35.20_imdb_val.csv",index_col=0)
imdb_summarized_test_val = pd.read_csv("./data/2. summarized/2024.12.25_06.10.20_imdb_test_val.csv",index_col=0)

In [2]:
import torch
device = 0 if torch.cuda.is_available() else -1
print(device)

0


##### Meta-Llama-3-8B-Instruct V1

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# 載入 模型 與 tokenizer 
model_or_path_name = "C:/Users/PARALELL/Desktop/code/DistilBert BERT/0. models/Meta-Llama-3-8B-Instruct"
model_Llama_3_8B_Instruct = AutoModelForCausalLM.from_pretrained(
    model_or_path_name,
    torch_dtype=torch.bfloat16,
    device_map=device,
)

tokenizer_Llama_3_8B_Instruct = AutoTokenizer.from_pretrained(model_or_path_name)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
# 啟用 xFormers 注意力計算（前提是模型和環境支援）
# model_Llama_3_8B_Instruct = model_Llama_3_8B_Instruct.to("cuda")
# model_Llama_3_8B_Instruct.config.use_cache = True
# model_Llama_3_8B_Instruct.config.use_xformers = True


def create_polarity_reversed_sentence(original_sentence):
    messages=[
            {
                "role": "system",
                "content": (
                    "You are tasked with reversing the sentiment of sentences. Transform positive sentences into negative ones, "
                    "and negative sentences into positive ones. Keep the main topic and structure of the original sentence unchanged. "
                    "Only modify the sentiment words and expressions.\n\n"
                    "Examples:\n"
                    "1. Original: I absolutely loved this movie! The acting was superb and the storyline was captivating.\n"
                    "   Rewritten: I absolutely hated this movie! The acting was terrible and the storyline was boring.\n"
                    "2. Original: The hotel was dirty and the staff were rude.\n"
                    "   Rewritten: The hotel was clean and the staff were friendly and accommodating.\n\n"
                    "Now reverse the sentiment of the following sentence while keeping its topic and structure intact:\n\n"
                    f"Original: {original_sentence}\nRewritten:"
                )
            },
            {
                "role": "user",
                "content": (
                    f"""{original_sentence}

    """
                    "Rewritten:"
                ),
            }
        ],

    if tokenizer_Llama_3_8B_Instruct.pad_token_id is None:
        tokenizer_Llama_3_8B_Instruct.pad_token_id = tokenizer_Llama_3_8B_Instruct.eos_token_id

    input_ids = tokenizer_Llama_3_8B_Instruct.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model_Llama_3_8B_Instruct.device)

    attention_mask = (input_ids != tokenizer_Llama_3_8B_Instruct.pad_token_id).long()

    terminators = [
        tokenizer_Llama_3_8B_Instruct.eos_token_id,
        tokenizer_Llama_3_8B_Instruct.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = model_Llama_3_8B_Instruct.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=1024,
        eos_token_id=terminators,
        pad_token_id=tokenizer_Llama_3_8B_Instruct.pad_token_id,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    
    response = outputs[0][input_ids.shape[-1]:]
    result = tokenizer_Llama_3_8B_Instruct.decode(response, skip_special_tokens=True)
    
    return result


In [5]:
def create_polarity_reversed_sentenceV2(original_sentence):
    messages = [
        {
            "role": "system",
            "content": (
                "You are tasked with reversing the sentiment of sentences. Transform positive sentences into negative ones, "
                "and negative sentences into positive ones. Keep the main topic and structure of the original sentence unchanged. "
                "Only modify the sentiment words and expressions.\n\n"
                "Examples:\n"
                "1. Original: I absolutely loved this movie! The acting was superb and the storyline was captivating.\n"
                "   Rewritten: I absolutely hated this movie! The acting was terrible and the storyline was boring.\n"
                "2. Original: The hotel was dirty and the staff were rude.\n"
                "   Rewritten: The hotel was clean and the staff were friendly and accommodating.\n\n"
                "Now reverse the sentiment of the following sentence while keeping its topic and structure intact:\n\n"
                f"Original: {original_sentence}\nRewritten:"
            )
        }
    ]

    if tokenizer_Llama_3_8B_Instruct.pad_token_id is None:
        tokenizer_Llama_3_8B_Instruct.pad_token_id = tokenizer_Llama_3_8B_Instruct.eos_token_id

    input_ids = tokenizer_Llama_3_8B_Instruct.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model_Llama_3_8B_Instruct.device)

    attention_mask = (input_ids != tokenizer_Llama_3_8B_Instruct.pad_token_id).long()

    terminators = [
        tokenizer_Llama_3_8B_Instruct.eos_token_id,
        tokenizer_Llama_3_8B_Instruct.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = model_Llama_3_8B_Instruct.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=1024,
        eos_token_id=terminators,
        pad_token_id=tokenizer_Llama_3_8B_Instruct.pad_token_id,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )

    response = outputs[0][input_ids.shape[-1]:]
    result = tokenizer_Llama_3_8B_Instruct.decode(response, skip_special_tokens=True)

    # 只提取反轉後的影評
    if "Rewritten:" in result:
        reversed_review = result.split("Rewritten:")[-1].strip()
    else:
        reversed_review = result.strip()

    return reversed_review


In [4]:
def create_polarity_reversed_sentenceV3(original_sentence):
    messages = [
        {
            "role": "system",
            "content": (
                "You are tasked with reversing the sentiment of sentences. Transform positive sentences into negative ones, "
                "and negative sentences into positive ones. Keep the main topic and structure of the original sentence unchanged. "
                "Only modify the sentiment words and expressions.\n\n"
                "Examples:\n"
                "1. Original: I absolutely loved this movie! The acting was superb and the storyline was captivating.\n"
                "   Rewritten: I absolutely hated this movie! The acting was terrible and the storyline was boring.\n"
                "2. Original: The hotel was dirty and the staff were rude.\n"
                "   Rewritten: The hotel was clean and the staff were friendly and accommodating.\n\n"
                "Now reverse the sentiment of the following sentence while keeping its topic and structure intact:\n\n"
                "Provide only the rewritten sentence without any additional text.\n\n"

                f"Original: {original_sentence}\nRewritten:"
            )
        }
    ]

    if tokenizer_Llama_3_8B_Instruct.pad_token_id is None:
        tokenizer_Llama_3_8B_Instruct.pad_token_id = tokenizer_Llama_3_8B_Instruct.eos_token_id

    input_ids = tokenizer_Llama_3_8B_Instruct.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model_Llama_3_8B_Instruct.device)

    attention_mask = (input_ids != tokenizer_Llama_3_8B_Instruct.pad_token_id).long()

    terminators = [
        tokenizer_Llama_3_8B_Instruct.eos_token_id,
        tokenizer_Llama_3_8B_Instruct.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = model_Llama_3_8B_Instruct.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=1024,
        eos_token_id=terminators,
        pad_token_id=tokenizer_Llama_3_8B_Instruct.pad_token_id,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )

    response = outputs[0][input_ids.shape[-1]:]
    result = tokenizer_Llama_3_8B_Instruct.decode(response, skip_special_tokens=True)

    # 只提取反轉後的影評
    if "Rewritten:" in result:
        reversed_review = result.split("Rewritten:")[-1].strip()
    else:
        reversed_review = result.strip()

    # 清理多餘內容，如 "Note:" 或其他無關部分
    cleaned_review = reversed_review.split("Note:")[0].strip()

    return cleaned_review


In [ ]:
summarizedReview = imdb_summarized_train["summarized_text"][248]
print(summarizedReview)
polarity_reversed_result = create_polarity_reversed_sentence(summarizedReview)
print(polarity_reversed_result)

In [5]:
# DistilBert
from transformers import pipeline, DistilBertForSequenceClassification, DistilBertTokenizerFast

model_name_or_path = "C:/Users/PARALELL/Desktop/code/DistilBert BERT/0. models/DistilBert/V2/MyBestIMDBModel2"

model_DistilBert_fine_tuned = DistilBertForSequenceClassification.from_pretrained(model_name_or_path,label2id={"NEG":0,"POS":1},id2label={0: "0", 1: "1"}) # ,id2label = {0:"NEG",1:"POS"}
tokenizer_DistilBert_fine_tuned = DistilBertTokenizerFast.from_pretrained(model_name_or_path)
model_DistilBert_fine_tuned.to(device)
nlp_DistilBert = pipeline("sentiment-analysis",model=model_DistilBert_fine_tuned,tokenizer=tokenizer_DistilBert_fine_tuned,device=device)

In [ ]:
result = nlp_DistilBert(polarity_reversed_result)
print(result)
sentimentLabel = result[0]["label"]
print("afterLabel :",sentimentLabel)

In [6]:
def get_polarity_reversed_sentiment(oneRowData):
    summarized_text = oneRowData["summarized_text"]
    summarized_sentiment = oneRowData["summarized_sentiment"]
    
    if (summarized_sentiment == "NEG"): # False
        summarized_sentiment = 0
    if (summarized_sentiment == "POS"): # Ture
        summarized_sentiment = 1
    print("summarized_sentiment :",summarized_sentiment)
    
    # 生成句子
    polarity_reversed_result = create_polarity_reversed_sentenceV3(summarized_text)
    print("-"*200)
    print(polarity_reversed_result)
    print("-"*200)
    # 取得生成後的情感分析結果
    sentimentresult = nlp_DistilBert(polarity_reversed_result)
    #print(result)
    sentimentLabel = sentimentresult[0]["label"]
    print("sentimentLabel :",sentimentLabel)
    sentimentScore = sentimentresult[0]["score"]
    print("sentimentScore :",sentimentScore)
    
    dict1 = {
            "summarized_text":summarized_text, # 摘要過後的影評
            "summarized_sentiment":summarized_sentiment, # 摘要後的影評的情感分析            
            "polarity_reversed_text":polarity_reversed_result,
            "polarity_reversed_sentiment_result":sentimentresult,
            "sentiment_label":sentimentLabel,
        }
    
    return dict1

In [ ]:
oneRowData = imdb_summarized_train.loc[0]
#print(oneRowData)
dict1 = get_polarity_reversed_sentiment(oneRowData)
print(dict1)

In [7]:
import pandas as pd
df1 = pd.DataFrame(columns=["summarized_text","summarized_sentiment","polarity_reversed_text","polarity_reversed_sentiment_result","sentiment_label"])
df1    

,summarized_text,summarized_sentiment,polarity_reversed_text,polarity_reversed_sentiment_result,sentiment_label


In [8]:
dataset = imdb_summarized_test_val

device = 0 if torch.cuda.is_available() else -1
# print("device :", device)

for i in range(231,dataset.shape[0]): # dataset.shape[0] # 231
    try:
        oneRowData = dataset.loc[i]    
        print("="*100)
        print(f"index = {i}")
        
        dict1 = get_polarity_reversed_sentiment(oneRowData)
        
        new_df = pd.DataFrame([dict1])
        df1 = pd.concat([df1, new_df], ignore_index=True)
        
    except Exception as error_msg:
        print(f"index = {i} 出現錯誤\n", error_msg)
        

index = 231
summarized_sentiment : 0


c:\Users\PARALELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\models\llama\modeling_llama.py:655: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
I remember rather disliking this a few years back but coming to it again, I'm surprised why. I guess it always looks good and the girls do rather well but the men do rather elevate the scene. Why oh why in so many English films about sex do we have to have such capable men alongside the pretty girls? This is both as innocently silly and as intelligently serious as it seems to intend. Richard O'Sullivan maybe, as such a central figure, could have helped but I reckon this to be one of his best performances.
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
sentimentLabel : 1
sentimentScore : 0.9390231370925903
index = 232
summarized_sentiment

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
All I can say after watching Snitch'd is please start Mr. Cahill. It's impressive to watch movies with realistic gangbangers that even sound more impressive when they speak. This movie was easy to watch, almost like an after school special. Snitch'd is the BEST gangbanger movie ever made. I suspect real gangbangers do wear hats claiming their gang. The gangbanger was from? That would be smart considering new laws that shorten a prison term.
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
sentimentLabel : 1
sentimentScore : 0.9913648366928101
index = 242
summarized_sentiment : 0
-------------------------------------------------------------

Token indices sequence length is longer than the specified maximum sequence length for this model (514 > 512). Running this sequence through the model will result in indexing errors


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Here is the rewritten sentence with the sentiment reversed:

Provide only the rewritten sentence without any additional text.

Masters of Horror: Right to Die starts late one night as married couple Abby (Julia Anderson) & Cliff Addison (Martin Donovan) are driving home. While talking Cliff is distracted & crashes into a tree that has fallen across the road. Cliff's airbag works OK & he walks away with minor injuries. Unfortunately for Cliff, his didn't & he ended up as toast when he was thrown from the car & doused in petrol. This American-Canadian co-production was directed by Rob Schmidt (whose only horror film previously was Wrong Turn (2003) This was episode 9 from season 2 of the Masters of Horror TV series, while I didn't watch it. The script by John Esposito has a neat central ide

In [9]:
df1

,summarized_text,summarized_sentiment,polarity_reversed_text,polarity_reversed_sentiment_result,sentiment_label
0,I remember rather enjoying this a few years ba...,0,I remember rather disliking this a few years b...,"[{'label': '1', 'score': 0.9390231370925903}]",1
1,"The ""saucy"" misadventures of four au pairs who...",0,"The ""innocent"" adventures of four au pairs who...","[{'label': '1', 'score': 0.9929803013801575}]",1
2,The plot has already been described by other r...,0,The plot has already been described by other r...,"[{'label': '1', 'score': 0.9901110529899597}]",1
3,The video quality is awful. The sound quality ...,0,The video quality is excellent. The sound qual...,"[{'label': '1', 'score': 0.9924485683441162}]",1
4,"I got this DVD from a friend, who got it from ...",0,"I got this DVD from a friend, who got it from ...","[{'label': '1', 'score': 0.9896925091743469}]",1
...,...,...,...,...,...
1730,Meryl Streep was incredible in this film. I re...,1,Meryl Streep was terrible in this film. I real...,"[{'label': '0', 'score': 0.9939374923706055}]",0
1731,Michael Kallio gives a strong and convincing p...,1,Michael Kallio gives a weak and unconvincing p...,"[{'label': '0', 'score': 0.995408833026886}]",0
1732,"""Hatred of a Minute"" is a psychological thrill...",1,"""Adoration of a Minute"" is a psychological thr...","[{'label': '1', 'score': 0.9918748736381531}]",1
1733,The best movie to come out of Michigan since.....,1,The worst movie to come out of Michigan since....,"[{'label': '0', 'score': 0.9940769672393799}]",0


In [ ]:
df1.loc[0]["polarity_reversed_text"]

In [10]:
from datetime import datetime
now = datetime.now()
formatted_time = now.strftime("%Y.%m.%d_%H.%M.%S")  # 格式：年-月-日 時:分:秒
#print("現在時間:", formatted_time)
fileName = f"./{formatted_time}_polarity_reversed_test_val_231_1735.csv"
df1.to_csv(fileName)